In [2]:
# In a Jupyter Notebook cell
%load_ext autoreload
%autoreload 2


In [3]:

import numpy as np
print(np.__version__)

from mirage import WordCountingChunkingAlgorithm, FolderRawStorage, WhooshChunkStorage
from mirage import HuggingFaceEmbedder, TextNormalizer
from mirage import NatashaSentenceChunking, SemanticChunking
import os
import pickle

1.26.4


c:\Users\Bulkin\MIRAGE\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
embedder = HuggingFaceEmbedder(model_name='BAAI/bge-m3', normalizer=True)

In [5]:
import logging
logging.basicConfig(level=logging.ERROR)


documents = FolderRawStorage('data_test')
chunks = WhooshChunkStorage(scoring_function="BM25F", normalizer=True)
# algorithm = WordCountingChunkingAlgorithm(documents, chunks, words_amount=100)


In [6]:

algorithm = SemanticChunking(raw_storage=documents, chunk_storage=chunks, embedder=embedder, threshold=0.5, max_chunk_size=1000)
algorithm.execute(visualize=True)

Splitting text into the sentences


611it [00:00, 31855.61it/s]


creating embeddings of the sentences for semantic grouping


100%|██████████| 611/611 [00:42<00:00, 14.28it/s]


Adding chunks to the storage


100%|██████████| 427/427 [00:02<00:00, 171.87it/s]


1

In [7]:
import random
index = chunks.get_indexes()[random.randint(1, 50)]
print(len(chunks[index]))
chunks[index]

152


'Пожав нам руки, он уже собирался усесться, как вдруг взор его упал на листок с забавными значками, который я только что рассматривал и оставил на столе.'

In [8]:
from mirage.index.vector_index.ram_vector_index import L2RAMVectorIndex
from mirage import FaissIndexFlatL2, FaissIndexIVFPQR
vector_index = FaissIndexIVFPQR(dimensionality=embedder.get_dimensionality())
embedder.convert_chunks_to_vector_index(chunks, vector_index, visualize=True)

Converting ChunkStorage to VectorIndex


100%|██████████| 427/427 [00:34<00:00, 12.46it/s]


In [11]:
query = 'убитая женщина'
query_vector = embedder.embed(text=query)
# print(query_vector)
answer = vector_index.query(
    query_vector, top_k=20
)
print([element.distance for element in answer])
[chunks[element.chunk_storage_key] for element in answer]

[0.8335791230201721, 0.8566122055053711, 0.8726161122322083, 0.8734219670295715, 0.8924689292907715, 0.8948429822921753, 0.929951012134552, 0.9746432304382324, 0.9803217649459839, 0.9977196455001831, 1.003746509552002, 1.0152220726013184, 1.01560640335083, 1.0271902084350586, 1.0438331365585327, 1.0458725690841675, 1.0471690893173218, 1.0567314624786377, 1.0579125881195068, 1.0834579467773438]


['— Она была найдена тяжело раненной возле своего мертвого мужа.',
 'Так рассказывают служанки. Он умер, она при смерти.',
 'Вы кончили тем, что убили этого благородного человека и довели его жену до самоубийства.',
 'Возле окна корчилась его жена, прислонясь головой к стене. Рана ее была ужасна — кровь залила половину лица.',
 'Их хозяин лежал посреди комнаты лицом вниз. Он был мертв.',
 'Моя жена получила письмо из Америки — на конверте была американская марка. Жена смертельно побледнела, прочла письмо и швырнула в огонь.',
 '— Если Илей умрет, мне все равно, что будет со мною, — сказал американец.',
 'Доктор удалился к своим пациентам.',
 'Я увидел, как из-за угла выползла темная согнутая фигура и уселась перед дверью. Схватив револьвер, я рванулся вперед, но жена судорожно обняла меня и удержала на месте. Я пытался оттолкнуть ее, но она вцепилась в меня еще отчаяннее.',
 'Он умер мгновенно и безболезненно.',
 '— Вы трогали убитого? — спросил Холме.',
 '— Его несомненно закрыла женщ

In [15]:
answer_fulltext = chunks.query("убийца")
answer_fulltext

[ChunkNote(text='Если этот Аб Слени, живущий у Элриджа, действительно убийца и если он удерет, пока я сижу здесь, меня ждут крупные неприятности.', raw_document_index='человечки.txt'),
 ChunkNote(text='В этом деле есть несколько ^пунктов, которые я еще не в состоянии вам разъяснить. Я в своих открытиях зашел уже так далеко, что будет благоразумнее, если я подожду еще немного, а потом объясню вам все сразу. — Как вам угодно, мистер Холме, лишь бы убийца не ушел от нас.', raw_document_index='человечки.txt')]

In [ ]:
from mirage.inference.prompters.APILLM import LLM
llm = LLM()
llm.do_request(query=query,
               chunk_storage=chunks,
               indexes=[element.chunk_storage_key for element in answer],
               prompt='')